In [120]:
#pip install numpy
#pip install pandas
#pip install requests
#pip install ipython
#если файл нужно открыть в юпитере , то перед каждым pip нужно поставить !
#библиотека concurrent.futures входит в стандартную библиотеку Python, начиная с версии 3.2 и ее не нужно устанавливать отдельно
#если версия питон ниже, то эту библиотеку нельзя будет использовать и код не будет работать , в этом случае можно открыть код в
#гугл коллабе (colab.google), там есть встроенные библиотеки, или обновить версию питон

import numpy as np
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from IPython.display import display

In [121]:
city1="Москва" #город отправления
city2="Волгоград" #город прибытия
date = "2025-03-10" #в кавычках нужно указать нужную дату , без кавычек будет ошибка
key = "d78ba33b-42fb-4720-9d5d-d3130fd08607" #мой апи ключ
transport="любой" # вид транспорта, если transport="любой", то будут показаны все виды транспорта( атвобус, самолет и поезд), если transport="автобус" , то только автобусы,если transport="самолет", то только самолеты, если transport="поезд", то только поезда
transfers="все" # наличие пересадок , если в кавычках написано все , то в таблице будут все варианты и с пересадками и без , если в кавычках написано да , то в таблице будут данные рейсов с пересадками, если transfers="нет", то в таблице будут только прямые рейсы
time="любая" # длительность поездки любая или если time=число (пример: time=4 ) это означает, что длительность поездки не дольше 4 часов


In [122]:
url_stations = "https://api.rasp.yandex.net/v3.0/stations_list/"
params_stations = {
        "apikey": key
    }
response_stations = requests.get(url_stations, params=params_stations)

In [123]:
stations = pd.json_normalize(response_stations.json(),
                        record_path=['countries', 'regions', 'settlements', 'stations'],
                        meta=[
                            ['countries', 'title'],
                            ['countries', 'codes'],
                            ['countries', 'regions', 'settlements', 'title'],
                        ],
                        errors='ignore'
                        )
stations = stations.replace('', np.nan)
stations.drop(['direction', 'codes.esr_code'], axis=1, inplace=True)
stations['countries.codes'] = stations['countries.codes'].apply(lambda x: x.get('yandex_code'))

<ipython-input-123-cf2883e9f81d>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stations = stations.replace('', np.nan)


In [124]:
if transport=="любой":
   stations = stations[stations["station_type"].isin(["airport", "train_station", "bus_station"])]
if transport=="автобус":
   stations = stations[stations["station_type"].isin(["bus_station"])]
if transport=="самолет":
   stations = stations[stations["station_type"].isin(["airport"])]
if transport=="поезд":
   stations = stations[stations["station_type"].isin(["train_station"])]

In [125]:
stat1=stations[stations["countries.regions.settlements.title"]==city1]
stat2=stations[stations["countries.regions.settlements.title"]==city2]

if len(stat1) == 0 or len(stat2) == 0:
    print("нет рейсов по выбранным параметрам , выберите другие ")

In [126]:
code1=stat1['codes.yandex_code'].tolist()
code2=stat2['codes.yandex_code'].tolist()

In [127]:
url_search = "https://api.rasp.yandex.net/v3.0/search/"

dataframes = []

def fetch_data(point1, point2):
    params_search = {
        "apikey": key,
        "from": point1,
        "to": point2,
        "date": date,
        "transfers": True
    }
    response_search = requests.get(url_search, params=params_search)

    if response_search.status_code == 404:
        return None
    elif response_search.json()['pagination']['total'] == 0:
        return None
    else:
        segments = pd.json_normalize(response_search.json(), record_path=['segments'],  errors='ignore')
        segments_important = segments.copy()

        columns = [
         'transport_types','has_transfers','from.transport_type','to.transport_type',
         'from.title','to.title','departure_from.title','arrival_to.title'
         ]

        for column in columns:
            if column not in segments_important.columns:
               segments_important[column] = np.nan



        if ('departure' in segments_important.columns and
            'arrival' in segments_important.columns):

            segments_important = segments_important[['departure', 'arrival','has_transfers' ,'from.transport_type','to.transport_type','transport_types',
             'from.title','to.title','departure_from.title','arrival_to.title']]

            segments_important['arrival'] = pd.to_datetime(segments_important['arrival'])
            segments_important['departure'] = pd.to_datetime(segments_important['departure'])
            segments_important['time_difference'] = segments_important['arrival'] - segments_important['departure']
            return segments_important
    return None


In [128]:
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_data, i, j) for i in code1 for j in code2]
    for future in futures:
        result = future.result()
        if result is not None:
            dataframes.append(result)

In [129]:
comb_dataframes = pd.concat(dataframes, ignore_index=True)
comb_dataframes=comb_dataframes.sort_values(by='time_difference', ascending=True)
comb_dataframes.head(60)
comb_dataframes['from.title'] = comb_dataframes['from.title'].combine_first(comb_dataframes['departure_from.title'])
comb_dataframes['to.title'] = comb_dataframes['to.title'].combine_first(comb_dataframes['arrival_to.title'])
comb_dataframes= comb_dataframes.drop(['departure_from.title','arrival_to.title'], axis=1)
comb_dataframes['comb'] = comb_dataframes.apply(lambda row: [row['from.transport_type'], row['to.transport_type']], axis=1)
comb_dataframes['transport_types'] = comb_dataframes['transport_types'].combine_first(comb_dataframes['comb'])
comb_dataframes= comb_dataframes.drop(['comb','from.transport_type','to.transport_type'], axis=1)
comb_dataframes.rename(columns={'departure': 'время отправления', 'arrival': 'время прибытия', 'has_transfers': 'наличие пересадок', 'transport_types': 'тип транспорта', 'from.title': 'пункт отправления', 'to.title': 'пункт прибытия', 'time_difference': 'время пути'}, inplace=True)
comb_dataframes=comb_dataframes.reset_index(drop=True)
comb_dataframes

,время отправления,время прибытия,наличие пересадок,тип транспорта,пункт отправления,пункт прибытия,время пути
0,2025-03-10 13:35:00+03:00,2025-03-10 15:30:00+03:00,False,"[plane, plane]",Домодедово,Гумрак,0 days 01:55:00
1,2025-03-10 08:45:00+03:00,2025-03-10 10:45:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
2,2025-03-10 14:30:00+03:00,2025-03-10 16:30:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
3,2025-03-10 18:25:00+03:00,2025-03-10 20:25:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
4,2025-03-10 18:30:00+03:00,2025-03-10 20:30:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
...,...,...,...,...,...,...,...
60,2025-03-10 23:45:00+03:00,2025-03-13 09:17:00+03:00,True,"[bus, train, bus]",Автовокзал Саларьево,"Волгоград, Центральный автовокзал",2 days 09:32:00
61,2025-03-10 01:39:00+03:00,2025-03-12 14:00:00+03:00,True,"[train, train, bus]",Москва (Белорусский вокзал),"Волгоград, Центральный автовокзал",2 days 12:21:00
62,2025-03-10 19:45:00+03:00,2025-03-13 09:51:00+03:00,True,"[train, train]",Москва (Киевский вокзал),Волгоград-1,2 days 14:06:00
63,2025-03-10 01:00:00+03:00,2025-03-12 18:53:00+03:00,True,"[train, train, bus]",Москва (Ярославский вокзал),"Волгоград, Центральный автовокзал",2 days 17:53:00


In [130]:
if transfers=="да":
   comb_dataframes=comb_dataframes[comb_dataframes['наличие пересадок']==True]
if transfers=="нет":
   comb_dataframes=comb_dataframes[comb_dataframes['наличие пересадок']==False]
if time!="любая":
   time_hour= timedelta(hours=time)
   comb_dataframes=comb_dataframes[comb_dataframes['время пути']<=time_hour]

pd.set_option('display.max_rows', None)  # Отображать все строки
pd.set_option('display.max_columns', None)

display(comb_dataframes)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


,время отправления,время прибытия,наличие пересадок,тип транспорта,пункт отправления,пункт прибытия,время пути
0,2025-03-10 13:35:00+03:00,2025-03-10 15:30:00+03:00,False,"[plane, plane]",Домодедово,Гумрак,0 days 01:55:00
1,2025-03-10 08:45:00+03:00,2025-03-10 10:45:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
2,2025-03-10 14:30:00+03:00,2025-03-10 16:30:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
3,2025-03-10 18:25:00+03:00,2025-03-10 20:25:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
4,2025-03-10 18:30:00+03:00,2025-03-10 20:30:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:00:00
5,2025-03-10 08:50:00+03:00,2025-03-10 10:55:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:05:00
6,2025-03-10 14:20:00+03:00,2025-03-10 16:25:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:05:00
7,2025-03-10 22:45:00+03:00,2025-03-11 00:50:00+03:00,False,"[plane, plane]",Шереметьево,Гумрак,0 days 02:05:00
8,2025-03-10 14:50:00+03:00,2025-03-11 07:40:00+03:00,True,"[train, train]",Москва (Казанский вокзал),Волгоград-1,0 days 16:50:00
9,2025-03-10 13:50:00+03:00,2025-03-11 07:40:00+03:00,False,"[train, train]",Москва (Павелецкий вокзал),Волгоград-1,0 days 17:50:00
